## Carregando documentos

In [1]:
import os.path
import glob 
import codecs
import nltk
import re
import spacy
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import unicodedata
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

In [3]:
corpus = ["""Naruto (ナルト?) é uma série de mangá escrita e ilustrada por Masashi Kishimoto, 
que conta a história de Naruto Uzumaki, um jovem ninja que constantemente procura por 
reconhecimento e sonha em se tornar Hokage, o ninja líder de sua vila. 
A história é dividida em duas partes, a primeira parte se passa nos anos da pré-adolescência de Naruto, 
e a segunda parte se passa em sua adolescência. A série é baseada em dois mangás one-shots de Kishimoto: 
Karakuri (1995), e que por causa dele Kishimoto ganhou uma menção honrosa no prêmio Hop Step da Shueisha 
no ano seguinte, e Naruto (1997). A editora Panini Comics licenciou e publicou o mangá em 
três versões no Brasil, e em Portugal é a editora Devir Manga quem licencia e publica o mangá. 
A história de Naruto continua com o seu filho, Boruto Uzumaki em Boruto: Naruto Next Generations.""",
     
"""Serial Experiments Lain (シリアルエクスペリメンツレイン Shiriaru Ekusuperimentsu Rein?) é uma série anime 
lançada pelo estúdio Triangle Staff, dirigida por Ryutaro Nakamura, criação de cenários por 
Chiaki J. Konaka e desenhada por Yoshitoshi ABe. A série foi transmitida pela TV Tokyo de Julho a 
Setembro de 1998 e teve um jogo para PlayStation lançado no mesmo ano.""",
     
"""Fullmetal Alchemist (鋼の錬金術師 Hagane no Renkinjutsushi?, lit. "Alquimista de Aço") 
é um anime baseado no mangá shōnen escrito e ilustrado por Hiromu Arakawa. O mangá foi
serializado na revista mensal japonesa Monthly Shōnen Gangan entre agosto de 2001 e junho de 2010, 
com os seus 108 capítulos individuais compilados em 27 volumes em formato tankōbon e publicados pela 
editora Square Enix. O mundo de Fullmetal Alchemist é baseado no período após a Revolução Industrial 
Europeia. Situado em um universo ficcional em que a alquimia é uma das mais avançadas técnicas científicas 
conhecidas pelo homem, a história centra-se nos irmãos Edward Elric e Alphonse Elric, que estão procurando 
a pedra filosofal para restaurar seus corpos após uma desastrosa tentativa de trazer a mãe falecida de volta à 
vida através da alquimia."""
]

In [4]:
len(corpus)

3

 # Pré-processamento

## limpando o texto

In [5]:
corpus_clean1 = [ unicodedata.normalize('NFKD', s).encode('ISO-8859-1', 'ignore').decode('ISO-8859-1') 
                 for s in corpus]

In [10]:
single_char = 'ê'
multiple_chars = '\N{LATIN SMALL LETTER E}\N{COMBINING CIRCUMFLEX ACCENT}'

In [13]:
print(single_char)
print(multiple_chars)

ê
ê


In [7]:
unicodedata.normalize('NFKD', corpus_clean1[0]).encode('ISO-8859-1', 'ignore')

b'Naruto (?) e uma serie de manga escrita e ilustrada por Masashi Kishimoto, \nque conta a historia de Naruto Uzumaki, um jovem ninja que constantemente procura por \nreconhecimento e sonha em se tornar Hokage, o ninja lider de sua vila. \nA historia e dividida em duas partes, a primeira parte se passa nos anos da pre-adolescencia de Naruto, \ne a segunda parte se passa em sua adolescencia. A serie e baseada em dois mangas one-shots de Kishimoto: \nKarakuri (1995), e que por causa dele Kishimoto ganhou uma mencao honrosa no premio Hop Step da Shueisha \nno ano seguinte, e Naruto (1997). A editora Panini Comics licenciou e publicou o manga em \ntres versoes no Brasil, e em Portugal e a editora Devir Manga quem licencia e publica o manga. \nA historia de Naruto continua com o seu filho, Boruto Uzumaki em Boruto: Naruto Next Generations.'

In [14]:
corpus_clean = [ re.sub(r'[^a-zA-Z ]',r' ',doc) for doc in corpus_clean1 ]

In [104]:
corpus_clean[0]

'Naruto     e uma serie de manga escrita e ilustrada por Masashi Kishimoto   que conta a historia de Naruto Uzumaki  um jovem ninja que constantemente procura por  reconhecimento e sonha em se tornar Hokage  o ninja lider de sua vila   A historia e dividida em duas partes  a primeira parte se passa nos anos da pre adolescencia de Naruto   e a segunda parte se passa em sua adolescencia  A serie e baseada em dois mangas one shots de Kishimoto   Karakuri         e que por causa dele Kishimoto ganhou uma mencao honrosa no premio Hop Step da Shueisha  no ano seguinte  e Naruto         A editora Panini Comics licenciou e publicou o manga em  tres versoes no Brasil  e em Portugal e a editora Devir Manga quem licencia e publica o manga   A historia de Naruto continua com o seu filho  Boruto Uzumaki em Boruto  Naruto Next Generations '

## stop words

In [17]:
stop_list = nltk.corpus.stopwords.words('portuguese')

In [19]:
print(stop_list)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

In [20]:
corpus_tokens = [nltk.word_tokenize(doc) for doc in corpus_clean]   

In [22]:
print(corpus_tokens[0])

['Naruto', 'e', 'uma', 'serie', 'de', 'manga', 'escrita', 'e', 'ilustrada', 'por', 'Masashi', 'Kishimoto', 'que', 'conta', 'a', 'historia', 'de', 'Naruto', 'Uzumaki', 'um', 'jovem', 'ninja', 'que', 'constantemente', 'procura', 'por', 'reconhecimento', 'e', 'sonha', 'em', 'se', 'tornar', 'Hokage', 'o', 'ninja', 'lider', 'de', 'sua', 'vila', 'A', 'historia', 'e', 'dividida', 'em', 'duas', 'partes', 'a', 'primeira', 'parte', 'se', 'passa', 'nos', 'anos', 'da', 'pre', 'adolescencia', 'de', 'Naruto', 'e', 'a', 'segunda', 'parte', 'se', 'passa', 'em', 'sua', 'adolescencia', 'A', 'serie', 'e', 'baseada', 'em', 'dois', 'mangas', 'one', 'shots', 'de', 'Kishimoto', 'Karakuri', 'e', 'que', 'por', 'causa', 'dele', 'Kishimoto', 'ganhou', 'uma', 'mencao', 'honrosa', 'no', 'premio', 'Hop', 'Step', 'da', 'Shueisha', 'no', 'ano', 'seguinte', 'e', 'Naruto', 'A', 'editora', 'Panini', 'Comics', 'licenciou', 'e', 'publicou', 'o', 'manga', 'em', 'tres', 'versoes', 'no', 'Brasil', 'e', 'em', 'Portugal', 'e',

In [27]:
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_list]  

corpus_stop = [' '.join(remove_stopwords(doc)) for doc in corpus_tokens]

In [30]:
print(corpus_stop[0])

Naruto serie manga escrita ilustrada Masashi Kishimoto conta historia Naruto Uzumaki jovem ninja constantemente procura reconhecimento sonha tornar Hokage ninja lider vila historia dividida duas partes primeira parte passa anos pre adolescencia Naruto segunda parte passa adolescencia serie baseada dois mangas one shots Kishimoto Karakuri causa Kishimoto ganhou mencao honrosa premio Hop Step Shueisha ano seguinte Naruto editora Panini Comics licenciou publicou manga tres versoes Brasil Portugal editora Devir Manga licencia publica manga historia Naruto continua filho Boruto Uzumaki Boruto Naruto Next Generations


## Stemming

In [32]:
corpus_tokens2 = [nltk.word_tokenize(doc) for doc in corpus_clean]   
ps = SnowballStemmer('portuguese')
def do_stem(tokens):
    return [ps.stem(word) for word in tokens]

corpus_stem = [' '.join(do_stem(doc)) for doc in corpus_tokens2]

In [33]:
corpus_stem[0]

'narut e uma seri de mang escrit e ilustr por masash kishimot que cont a histor de narut uzumak um jov ninj que constant procur por reconhec e sonh em se torn hokag o ninj lid de sua vil a histor e divid em duas part a primeir part se pass nos anos da pre adolescenc de narut e a segund part se pass em sua adolescenc a seri e bas em dois mang one shots de kishimot karakur e que por caus del kishimot ganh uma menca honros no premi hop step da shueish no ano seguint e narut a editor panin comics licenc e public o mang em tres verso no brasil e em portugal e a editor dev mang quem licenc e public o mang a histor de narut continu com o seu filh borut uzumak em borut narut next generations'

In [111]:
corpus_stem[0]

'narut e uma seri de mang escrit e ilustr por masash kishimot que cont a histor de narut uzumak um jov ninj que constant procur por reconhec e sonh em se torn hokag o ninj lid de sua vil a histor e divid em duas part a primeir part se pass nos anos da pre adolescenc de narut e a segund part se pass em sua adolescenc a seri e bas em dois mang one shots de kishimot karakur e que por caus del kishimot ganh uma menca honros no premi hop step da shueish no ano seguint e narut a editor panin comics licenc e public o mang em tres verso no brasil e em portugal e a editor dev mang quem licenc e public o mang a histor de narut continu com o seu filh borut uzumak em borut narut next generations'

## Lematizador


In [34]:
# sp = spacy.load('en_core_web_sm', disable=['parser','ner'])
#sp = spacy.load('en_core_web_sm')
sp = spacy.load("pt_core_news_sm")

In [35]:
doc = sp(corpus[0])
' '.join([token.lemma_ for token in doc])

'Naruto ( ナルト ? ) ser umar série de mangar escrito e ilustrar por Masashi Kishimoto , \n que contar o história de Naruto Uzumaki , um jovem ninja que constantemente procurar por \n reconhecimento e sonhar em se tornar Hokage , o ninja líder de suar vila . \n A história ser dividir em dois partir , o primeiro partir se passar o ano da pré-adolescência de Naruto , \n e o segundo partir se passar em suar adolescência . A série ser basear em dois mangás one-shots de Kishimoto : \n Karakuri ( 1995 ) , e que por causar d ele Kishimoto ganhar umar menção honroso o prêmio Hop Step da Shueisha \n o ano seguinte , e Naruto ( 1997 ) . A editorar Panini Comics licenciar e publicar o mangar em \n três versão o Brasil , e em Portugal ser o editorar Devir Manga quem licenciar e publicar o mangar . \n A história de Naruto continuar com o seu filhar , Boruto Uzumaki em Boruto : Naruto Next Generations .'

In [38]:
print(corpus[0])

Naruto (ナルト?) é uma série de mangá escrita e ilustrada por Masashi Kishimoto, 
que conta a história de Naruto Uzumaki, um jovem ninja que constantemente procura por 
reconhecimento e sonha em se tornar Hokage, o ninja líder de sua vila. 
A história é dividida em duas partes, a primeira parte se passa nos anos da pré-adolescência de Naruto, 
e a segunda parte se passa em sua adolescência. A série é baseada em dois mangás one-shots de Kishimoto: 
Karakuri (1995), e que por causa dele Kishimoto ganhou uma menção honrosa no prêmio Hop Step da Shueisha 
no ano seguinte, e Naruto (1997). A editora Panini Comics licenciou e publicou o mangá em 
três versões no Brasil, e em Portugal é a editora Devir Manga quem licencia e publica o mangá. 
A história de Naruto continua com o seu filho, Boruto Uzumaki em Boruto: Naruto Next Generations.


# Criando Matrizes

## Matriz Esparsa

In [39]:
from scipy.sparse import csr_matrix
import numpy as np

indptr = np.array([0,2,3,6])
indices = np.array([0,2,2,0,1,2])
data = np.array([1,2,3,4,5,6])
S = csr_matrix( (data,indices,indptr))
print(S)

  (0, 0)	1
  (0, 2)	2
  (1, 2)	3
  (2, 0)	4
  (2, 1)	5
  (2, 2)	6


In [41]:
S.todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

## Criando matrizes

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [44]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(corpus)
X.shape

(3, 194)

In [49]:
names = count_vect.get_feature_names()

In [50]:
X.getrow(0).nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([115, 192, 182, 169,  43, 104,  59,  82, 132, 106,  93, 142,  37,
         77, 184, 181,  89, 117,  36, 135, 144, 164,  56, 151, 174,  78,
        101, 168, 187,  49,  51, 125, 134, 124, 126, 119,  16,  41, 137,
          8, 153,  22,  50, 105, 120, 160,  92,   1,  27,  44,  73, 108,
         80, 118, 138,  81, 167, 161,  15, 152,   2,  52, 122,  33,  99,
        141, 178, 185,  25, 133,  47, 103, 143,  98, 139,  38,  32, 157,
         67,  24, 116,  74], dtype=int32))

In [52]:
print( [ (names[w],w) for w in X.getrow(0).nonzero()[1]] )

[('naruto', 115), ('ナルト', 192), ('uma', 182), ('série', 169), ('de', 43), ('mangá', 104), ('escrita', 59), ('ilustrada', 82), ('por', 132), ('masashi', 106), ('kishimoto', 93), ('que', 142), ('conta', 37), ('história', 77), ('uzumaki', 184), ('um', 181), ('jovem', 89), ('ninja', 117), ('constantemente', 36), ('procura', 135), ('reconhecimento', 144), ('sonha', 164), ('em', 56), ('se', 151), ('tornar', 174), ('hokage', 78), ('líder', 101), ('sua', 168), ('vila', 187), ('dividida', 49), ('duas', 51), ('partes', 125), ('primeira', 134), ('parte', 124), ('passa', 126), ('nos', 119), ('anos', 16), ('da', 41), ('pré', 137), ('adolescência', 8), ('segunda', 153), ('baseada', 22), ('dois', 50), ('mangás', 105), ('one', 120), ('shots', 160), ('karakuri', 92), ('1995', 1), ('causa', 27), ('dele', 44), ('ganhou', 73), ('menção', 108), ('honrosa', 80), ('no', 118), ('prêmio', 138), ('hop', 81), ('step', 167), ('shueisha', 161), ('ano', 15), ('seguinte', 152), ('1997', 2), ('editora', 52), ('panini

In [53]:
tfidf_vect = TfidfVectorizer()
X2 = tfidf_vect.fit_transform(corpus)
X2.shape

(3, 194)

In [57]:
hash_vect = HashingVectorizer(n_features=10000)
X3 = hash_vect.fit_transform(corpus)
X3.shape

(3, 10000)

In [60]:
hash_vect.vocabulary_

AttributeError: 'HashingVectorizer' object has no attribute 'vocabulary_'

In [59]:
count_vect.vocabulary_['about'], tfidf_vect.vocabulary_['about'], hash_vect.vocabulary_['about']

AttributeError: 'HashingVectorizer' object has no attribute 'vocabulary_'

# Preprocessando com scikit-learn

In [61]:
# carregando os dados
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc',
               'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes')
                                       , categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes')
                                       , categories=categories)


In [68]:
newsgroups_test.target_names

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']

In [69]:
count_vect = TfidfVectorizer()
X_train = count_vect.fit_transform(newsgroups_train.data)
print(X_train.shape)

tfidf_vect = TfidfVectorizer()
X_test = tfidf_vect.fit_transform(newsgroups_test.data)
print(X_test.shape)

(2034, 26879)
(1353, 21544)


## limpando o texto

In [86]:
tfidf_vect = TfidfVectorizer(strip_accents='unicode', lowercase=True, )
X2 = tfidf_vect.fit_transform(corpus)
print(X2.shape)

(299, 8372)


In [70]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vect = TfidfVectorizer(tokenizer=LemmaTokenizer(), strip_accents='unicode')
X2 = vect.fit_transform(newsgroups_train.data)
print(X2.shape)

(2034, 29277)


In [71]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.corpus import stopwords


def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = ["#NUMBER" if token[0].isdigit() else token for token in words]
    words = [porter_stemmer.stem(word) for word in words]
    return words

stem_stopwords = stemming_tokenizer(' '.join(stopwords.words('english')))
vect = TfidfVectorizer(stop_words = stem_stopwords, tokenizer=stemming_tokenizer, strip_accents='unicode')
X2 = vect.fit_transform(newsgroups_train.data)
print(X2.shape)

/home/thiagodepaulo/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(2034, 19081)


In [72]:
vect.inverse_transform(X2)[0]

array(['rych', 'somewher', 'avail', 'cel', 'format', 'read', 'abl',
       'like', 'rule', 'textur', 'manual', 'said', 'explicitli', 'noth',
       'store', 'inform', 'know', 'anyon', 'preserv', 'prj', 'orient',
       'default', 'given', 'restart', 'reload', 'file', '#number', 'care',
       'posit', 'plane', 'map', 'model', 'save', 'notic'], dtype='<U83')

In [73]:
newsgroups_train.data[0]

"Hi,\n\nI've noticed that if you only save a model (with all your mapping planes\npositioned carefully) to a .3DS file that when you reload it after restarting\n3DS, they are given a default position and orientation.  But if you save\nto a .PRJ file their positions/orientation are preserved.  Does anyone\nknow why this information is not stored in the .3DS file?  Nothing is\nexplicitly said in the manual about saving texture rules in the .PRJ file. \nI'd like to be able to read the texture rule information, does anyone have \nthe format for the .PRJ file?\n\nIs the .CEL file format available from somewhere?\n\nRych"

In [84]:
vect = CountVectorizer(max_df = 0.3, stop_words = stem_stopwords, tokenizer=stemming_tokenizer, )
X2 = vect.fit_transform(newsgroups_train.data)
print(X2.shape)

(2034, 19077)


In [85]:
vect.stop_words_

{'#number'}

In [91]:
vect = TfidfVectorizer(min_df=2, stop_words = stem_stopwords, tokenizer=stemming_tokenizer, )
X2 = vect.fit_transform(newsgroups_train.data)
print(X2.shape)

(2034, 8408)


In [97]:
vect = TfidfVectorizer(ngram_range=(1,2), stop_words = stem_stopwords, tokenizer=stemming_tokenizer, )
X2 = vect.fit_transform(newsgroups_train.data)
print(X2.shape)

(2034, 181266)


KeyError: 0

## Criando Pipeline

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier


pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),    
}

scoring = {
        'accuracy',
        'f1_micro',
        'f1_macro',
        'f1_weighted'
    }

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, 
                           return_train_score=False, scoring=scoring, refit='f1_macro')
grid_search.fit(newsgroups_train.data, newsgroups_train.target)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.5min finished
/home/thiagodepaulo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': (1e-05, 1e-06), 'clf__max_iter': (20,),
                         'clf__penalty': ('l2', 'elasticnet'),
                         'tfidf__norm': ('l1', 'l2'),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             refit='f1_macro',
             scoring={'f1_weighted', 'f1_micro', 'f1_macro', 'accuracy'},
             verbose=1)

In [101]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_clf__max_iter,param_clf__penalty,param_tfidf__norm,param_vect__max_df,param_vect__ngram_range,...,std_test_f1_macro,rank_test_f1_macro,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,1.781466,0.075570,0.218755,0.040950,1e-05,20,l2,l1,0.5,"(1, 1)",...,0.006572,3,0.830467,0.835381,0.842752,0.842752,0.839901,0.838251,0.004734,1
1,2.730455,0.084023,0.310166,0.081767,1e-05,20,l2,l1,0.5,"(1, 2)",...,0.021929,46,0.815725,0.788698,0.783784,0.820639,0.790640,0.799897,0.015176,45
2,0.762487,0.178918,0.104586,0.013136,1e-05,20,l2,l1,0.75,"(1, 1)",...,0.005963,11,0.835381,0.840295,0.835381,0.830467,0.834975,0.835300,0.003112,6
3,2.709784,0.205260,0.283460,0.046596,1e-05,20,l2,l1,0.75,"(1, 2)",...,0.010832,43,0.805897,0.808354,0.791155,0.808354,0.812808,0.805313,0.007422,43
4,0.803316,0.104262,0.138345,0.011105,1e-05,20,l2,l1,1,"(1, 1)",...,0.007361,12,0.830467,0.842752,0.835381,0.832924,0.832512,0.834807,0.004268,8
5,3.331783,0.293534,0.428598,0.069150,1e-05,20,l2,l1,1,"(1, 2)",...,0.019061,45,0.810811,0.800983,0.791155,0.815725,0.780788,0.799892,0.012747,46
6,0.844691,0.156000,0.227587,0.030258,1e-05,20,l2,l2,0.5,"(1, 1)",...,0.005049,27,0.828010,0.810811,0.825553,0.825553,0.817734,0.821532,0.006382,31
7,3.293817,0.251922,0.380334,0.088700,1e-05,20,l2,l2,0.5,"(1, 2)",...,0.007373,23,0.830467,0.828010,0.818182,0.830467,0.812808,0.823987,0.007195,25
8,1.004079,0.047236,0.191473,0.037258,1e-05,20,l2,l2,0.75,"(1, 1)",...,0.011355,14,0.823096,0.840295,0.828010,0.813268,0.830049,0.826944,0.008842,17
9,3.313298,0.113640,0.450777,0.171966,1e-05,20,l2,l2,0.75,"(1, 2)",...,0.006378,6,0.825553,0.835381,0.828010,0.840295,0.837438,0.833335,0.005628,10


In [102]:
grid_search.best_estimator_ , grid_search.best_score_

(Pipeline(steps=[('vect', CountVectorizer(max_df=0.5)),
                 ('tfidf', TfidfTransformer(norm='l1')),
                 ('clf', SGDClassifier(alpha=1e-06, max_iter=20))]),
 0.8247948617768188)

In [103]:
from estimator_select import EstimatorSelectionHelper
from sklearn.ensemble import (ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

vect = TfidfVectorizer()
X2 = vect.fit_transform(newsgroups_train.data)

models1 = {
        'ExtraTreesClassifier': ExtraTreesClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'AdaBoostClassifier': AdaBoostClassifier(),
        'GradientBoostingClassifier': GradientBoostingClassifier(),
        'SVC': SVC(),
        'MultinomialNB': MultinomialNB(),
        'KNN': KNeighborsClassifier()
    }

params1 = {
        'ExtraTreesClassifier': {'n_estimators': [16, 32]},
        'RandomForestClassifier': {'n_estimators': [16, 32]},
        'AdaBoostClassifier': {'n_estimators': [16, 32]},
        'GradientBoostingClassifier': {'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0]},
        'SVC': [
            {'kernel': ['linear'], 'C': [1, 10]},
            {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001]},
        ],
        'MultinomialNB': {'alpha': [0.1, 0.01, 0.001]},
        'KNN': {'n_neighbors': [1,3,5,7], 'metric': ['cosine'] }
    }

scoring = {
        'accuracy',
        'f1_micro',
        'f1_macro',
        'f1_weighted'
    }

helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X2, newsgroups_train.target, scoring=scoring, n_jobs=-1, cv=10)


Running GridSearchCV for ExtraTreesClassifier.
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for RandomForestClassifier.
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for AdaBoostClassifier.
Fitting 10 folds for each of 2 candidates, totalling 20 fits


...

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for MultinomialNB.
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for KNN.
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.7s finished


In [216]:
# fix it, writ in a file
helper1.score_summary(sort_by='min_score')


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_f1_micro,split1_test_f1_micro,split2_test_f1_micro,split3_test_f1_micro,...,std_test_f1_macro,rank_test_f1_macro,estimator,param_learning_rate,param_C,param_kernel,param_gamma,param_alpha,param_metric,param_n_neighbors
0,1.462394,0.285814,0.019146,0.002338,16,{'n_estimators': 16},0.730392,0.710784,0.715686,0.671569,...,0.025448,2,ExtraTreesClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.666805,0.556729,0.016941,0.002848,32,{'n_estimators': 32},0.750000,0.720588,0.740196,0.740196,...,0.024955,1,ExtraTreesClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.956224,0.122629,0.011058,0.000535,16,{'n_estimators': 16},0.622549,0.647059,0.622549,0.647059,...,0.033185,2,RandomForestClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.481530,0.046704,0.016678,0.001770,32,{'n_estimators': 32},0.725490,0.720588,0.705882,0.696078,...,0.025999,1,RandomForestClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1.869839,0.164228,0.016074,0.003906,16,{'n_estimators': 16},0.583333,0.509804,0.490196,0.563725,...,0.032408,2,AdaBoostClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.342742,0.988452,0.029473,0.009303,32,{'n_estimators': 32},0.656863,0.573529,0.553922,0.549020,...,0.039440,1,AdaBoostClassifier,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,15.776915,0.405385,0.010098,0.000645,16,"{'learning_rate': 0.8, 'n_estimators': 16}",0.715686,0.676471,0.661765,0.705882,...,0.034531,2,GradientBoostingClassifier,0.8,NaN,NaN,NaN,NaN,NaN,NaN
1,32.970417,1.218046,0.010846,0.000849,32,"{'learning_rate': 0.8, 'n_estimators': 32}",0.671569,0.710784,0.696078,0.715686,...,0.035063,1,GradientBoostingClassifier,0.8,NaN,NaN,NaN,NaN,NaN,NaN
2,17.689266,0.444532,0.011198,0.001233,16,"{'learning_rate': 1.0, 'n_estimators': 16}",0.700980,0.681373,0.637255,0.700980,...,0.026341,4,GradientBoostingClassifier,1,NaN,NaN,NaN,NaN,NaN,NaN
3,29.268113,5.704844,0.009681,0.001979,32,"{'learning_rate': 1.0, 'n_estimators': 32}",0.715686,0.651961,0.666667,0.705882,...,0.028321,3,GradientBoostingClassifier,1,NaN,NaN,NaN,NaN,NaN,NaN
